In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Generate 24 random full names
first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]
random.seed(0)  # For reproducibility
full_names = [random.choice(first_names) + " " + random.choice(last_names) for _ in range(24)]

# Generate a random conflict matrix with 40 conflicts for 24 people
np.random.seed(0)  # For reproducibility
conflicts = np.zeros((24, 24))
num_conflicts = 40

while num_conflicts > 0:
    i, j = np.random.randint(0, 24, size=2)
    if i != j and conflicts[i][j] == 0:
        conflicts[i][j] = 1
        conflicts[j][i] = 1
        num_conflicts -= 1
conflicts = torch.tensor(conflicts)

# Define the neural network with an additional layer
class SeatingArrangementNN(nn.Module):
    def __init__(self):
        super(SeatingArrangementNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(24, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 24),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

# Instantiate the model
model = SeatingArrangementNN()

# Define the loss function and optimizer with a smaller learning rate
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

def custom_loss(output, conflicts):
    # Initialize loss with a small base value to prevent zero gradients
    loss = torch.tensor(0.1, requires_grad=True)
    # Add MSE loss for each person to be seated (output close to 1)
    for i in range(24):
        loss = loss + criterion(output[i], torch.tensor(1.0, requires_grad=True))
    # Add penalty for conflicts
    for i in range(24):
        for j in range(i+1, 24):  # Check one half due to symmetry
            if conflicts[i][j] == 1:
                # Penalize if conflicting people are seated near each other
                if abs(i - j) == 1:
                    loss = loss + output[i] * output[j] * 10  # Weighted penalty for conflicts
    return loss


# Example input tensor (flat representation of seating arrangement)
input_tensor = torch.rand((1, 24), requires_grad=True)

# Training loop with increased number of epochs
for epoch in range(200):
    optimizer.zero_grad()
    output = model(input_tensor)
    loss = custom_loss(output.squeeze(), conflicts)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Output seating arrangement (example)
seating_arrangement = output.detach().squeeze().numpy()
print("Seating arrangement:", seating_arrangement)


Epoch 1, Loss: 11.05204963684082
Epoch 11, Loss: 10.62156867980957
Epoch 21, Loss: 10.18160629272461
Epoch 31, Loss: 9.61439323425293
Epoch 41, Loss: 8.8356294631958
Epoch 51, Loss: 7.821688175201416
Epoch 61, Loss: 6.643749237060547
Epoch 71, Loss: 5.491575241088867
Epoch 81, Loss: 4.612563610076904
Epoch 91, Loss: 4.082701683044434
Epoch 101, Loss: 3.802276611328125
Epoch 111, Loss: 3.6402907371520996
Epoch 121, Loss: 3.5190818309783936
Epoch 131, Loss: 3.4043757915496826
Epoch 141, Loss: 3.2824223041534424
Epoch 151, Loss: 3.1464197635650635
Epoch 161, Loss: 2.9903905391693115
Epoch 171, Loss: 2.811936855316162
Epoch 181, Loss: 2.6186411380767822
Epoch 191, Loss: 2.4410617351531982
Seating arrangement: [0.98210365 0.9666282  0.97272915 0.96996754 0.97278655 0.98291266
 0.01360748 0.7128501  0.9856793  0.00472259 0.83092636 0.98864824
 0.98846835 0.9853254  0.9782234  0.9757848  0.97603214 0.97127354
 0.97809416 0.97489357 0.9772084  0.9697911  0.9605141  0.9790684 ]
